In [1]:
from keras.layers import Activation, Dropout, Dense, Input, Add, Multiply, Concatenate,Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D,  Flatten, Dot,Reshape
from keras.models import Model
import random, time, os
import numpy as np
from keras import losses
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python import keras

def create_sample_size_dataset(all_ipds, sample_size, n_sample):
    #number_of_samples = int(len(all_ipds) / sample_size)
    all_samples = []
    for p in range(n_sample):
        all_samples.append(all_ipds[p * sample_size:(p + 1) * sample_size])
    return all_samples

def write_array_to_file(array, target, delimiter):
    for k in range(0, len(array)):
        target.write(str(array[k]) + delimiter)
    target.write("\n")

def read_from_file(path):
    with open(path, 'r') as content_file:
        content = content_file.read()
        return content


def create_ipd_dataset(address):
    files = os.listdir(address)
    all_ipds = []
    for f in files:
            ipd = read_from_file(address + f).split(' ')
            all_ipds.extend(convert_stringArrays_to_floatArray(ipd))
    return all_ipds


def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def convert_stringArrays_to_floatArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(float(k))
    return intArray


def convert_stringArrays_to_intArray(array):
    intArray = []

    for k in array:
        if isfloat(k):
            intArray.append(int(k))
    return intArray

def get_fingerprints_for_ipds(n_train, sample_size, alpha):
    
    #Previous one was all + and the largest value was 50.
   
    fingerprint_output = []
    while len(fingerprint_output) < n_train:
        finger = [random.uniform(0, 250)]
        neg_numbers = 0
        for i in range(sample_size - 1):
            #if random.randrange(0, 3) == 0:
            if random.randrange(0, 2) == 1:
                finger.append(random.uniform(0, alpha))#random.uniform(0, 10)
            else:
                finger.append(-1 * random.uniform(0, alpha))#
            if sum(finger) < 0:
                neg_numbers += 1
#             else:
#                 finger.append(0) 
        #if neg_numbers < 50:  ## this can be a hyperparameter
        fingerprint_output.append(finger)
    return fingerprint_output

def get_keys_for_fingerprinting_data(size, key_options):
    selected_keys = []
    for i in range(size):
        rnd = random.randrange(0, len(key_options))
        selected_keys.append(key_options[rnd])

    return selected_keys


def get_false_true_data(X, key_options,alpha):
    
    training_keys_true = get_keys_for_fingerprinting_data(size=len(X), key_options=key_options)
    #X_train_true =X# [x for x in X]
    y_train_true = get_fingerprints_for_ipds(len(X), sample_size=len(X[0]),alpha=alpha)#,training_keys=training_keys_true)
    
    ####### changing true trianing to false
    
    X_train = np.expand_dims(X, axis=1).reshape((-1, len(X[0]), 1))
    y_train = np.expand_dims(y_train_true, axis=1).reshape((-1, len(X[0]), 1))
    training_keys = training_keys_true#,#np.expand_dims(training_keys_true, axis=1)
    
    return X_train, y_train, training_keys

def selecting_valid_fingerprints(key_length):
    all_keys = []
    #address = '/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/keys/' + str(key_length) + "/"
    key_i = np.zeros(key_length)
    #keys = os.listdir(address)
    for k in range(key_length):
        key_i[k] = 1
        #key_i = convert_stringArrays_to_intArray(read_from_file(address + k).split(" "))
        #if key_i [0] == 1:
         #   continue
        all_keys.append(key_i)
        key_i[k] = 0
            
    return all_keys



Using TensorFlow backend.


In [2]:
def get_encoder_decoder_conv_dense_slice(sample_size, key_length, chunk):
    p = 0
    Input_ipd = Input(shape=(sample_size, 1), name='input1')  # this is needed just for the decoding
    Input_key = Input(shape=(key_length,), name='input2')
    fingerprint_mult = Input(shape=(chunk,), name='input3')
    fingerprint_sub = Input(shape=(chunk,), name='input4')
    network_noise = Input(shape=(sample_size,), name='input5')
    
    ipd = Flatten(name ="ipd_flatten1")(Input_ipd)
    outputs = []
    
    quant = int(sample_size/chunk)
    def slice(x):
        return x[:, p * chunk:(1 + p) * chunk]
    
    key1 = Dense(64, name='key1')(Input_key)

    sliced_ipd = Lambda(slice)(ipd)
    x_fingerprint = sliced_ipd
    for i in range(0, quant):
        sliced_ipd = Lambda(slice)(ipd)
        ss = Concatenate(name = 'concat'+ str(p))([x_fingerprint, sliced_ipd]) 
        ipd1 = Dense(32, name = 'dense'+ str(p))(ss)
        batch_2 = BatchNormalization(name = 'batch'+ str(p))(ipd1)
        relu_2 = Activation('relu', name = 'act'+ str(p))(batch_2)
        
        ipds_key_merge = Concatenate(name = 'concat_key_'+ str(p))([relu_2, key1])
        dense_enc1 = Dense(64, name = 'dense_enc1' + str(p))(ipds_key_merge)
        batch_2 = BatchNormalization(name = 'batch2_'+ str(p))(dense_enc1)
        relu_2 = Activation('relu', name = 'act2_'+ str(p))(batch_2)
        dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)
        
        x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
        x_fingerprint_mult = Multiply(name = 'fingerprint_mult' + str(p))([x_fingerprint_sig, fingerprint_mult])
        x_fingerprint = Add(name = 'ipd_delay' + str(p))([x_fingerprint_mult, fingerprint_sub])
        outputs.append(x_fingerprint)
        p += 1
    x_fingerprint = Concatenate(name = 'fingerprint2')(outputs)
    x_fingerprint_output = Reshape((sample_size, 1), name='fingerprint')(x_fingerprint)

    x_ipd = Add(name = 'x_ipd')([x_fingerprint, ipd, network_noise])
        
    x_ipd_reshape = Reshape((sample_size, 1),name = 'reshape_dec')(x_ipd)
    
    conv_dec_2 = Conv1D(filters = 20, kernel_size=10, padding='same', name='conv_dec_2')(x_ipd_reshape)
    conv_batch_2 = BatchNormalization(name='conv_batch_2_dec')(conv_dec_2)
    conv_relu_2 = Activation('relu', name='conv_relu_2_dec')(conv_batch_2)
    conv_drop_2 = Dropout(0.3, name='conv_drop_2_dec')(conv_relu_2)
    max_pool_dec_2 = MaxPooling1D(pool_size=1, name="max_pool_dec_2")(conv_drop_2)
    
    conv_dec_3 = Conv1D(filters = 10, kernel_size=10, padding='same', name='conv_dec_3')(max_pool_dec_2)
    conv_batch_3 = BatchNormalization(name='conv_batch_3_dec')(conv_dec_3)
    conv_relu_3 = Activation('relu', name='conv_relu_3_dec')(conv_batch_3)
    conv_drop_2 = Dropout(0.3, name='conv_drop_3_dec')(conv_relu_3)
    max_pool_dec_3 = MaxPooling1D(pool_size=1, name="max_pool_dec_3")(conv_drop_2)
    max_pool_dec_3_f = Flatten(name ="flate_max3_dec")(max_pool_dec_3)

    dense_dec_1 = Dense(256, name='dense_dec_1')(max_pool_dec_3_f)
    
    dense_batch_dec1 = BatchNormalization(name='dense_batch_dec1')(dense_dec_1)
    dense_relu_dec1 = Activation('relu', name='dense_relu_dec1')(dense_batch_dec1)
    dense_drop_dec1 = Dropout(0.3, name='dense_drop_dec1')(dense_relu_dec1)    
    
    dense_dec_2 = Dense(64, name='dense_dec_2')(dense_drop_dec1)
    dense_batch_dec2 = BatchNormalization(name='dense_batch_dec2')(dense_dec_2)
    dense_relu_dec2 = Activation('relu', name='dense_relu_dec2')(dense_batch_dec2)
    dense_drop_dec2 = Dropout(0.3, name='dense_drop_dec2')(dense_relu_dec2)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(dense_drop_dec2)

    return Model(inputs=[Input_ipd, Input_key, fingerprint_mult, fingerprint_sub, network_noise], outputs=[x_fingerprint_output, key_hat])#, key_hat])


def load_decoder(key_length, sample_size):
    
    Input_ipd = Input(shape=(sample_size, 1), name='reshape_dec') 
    #x_ipd_reshape = Reshape((sample_size, 1), name = 'reshape')(Input_ipd)
    conv_dec_2 = Conv1D(filters = 20, kernel_size=10, padding='same', name='conv_dec_2')(Input_ipd)
    conv_batch_2 = BatchNormalization(name='conv_batch_2_dec')(conv_dec_2)
    conv_relu_2 = Activation('relu', name='conv_relu_2_dec')(conv_batch_2)
    conv_drop_2 = Dropout(0.3, name='conv_drop_2_dec')(conv_relu_2)
    max_pool_dec_2 = MaxPooling1D(pool_size=1, name="max_pool_dec_2")(conv_drop_2)
    
    conv_dec_3 = Conv1D(filters = 10, kernel_size=10, padding='same', name='conv_dec_3')(max_pool_dec_2)
    conv_batch_3 = BatchNormalization(name='conv_batch_3_dec')(conv_dec_3)
    conv_relu_3 = Activation('relu', name='conv_relu_3_dec')(conv_batch_3)
    conv_drop_2 = Dropout(0.3, name='conv_drop_3_dec')(conv_relu_3)
    max_pool_dec_3 = MaxPooling1D(pool_size=1, name="max_pool_dec_3")(conv_drop_2)
    max_pool_dec_3_f = Flatten(name ="flate_max3_dec")(max_pool_dec_3)

    dense_dec_1 = Dense(256, name='dense_dec_1')(max_pool_dec_3_f)
    dense_batch_dec1 = BatchNormalization(name='dense_batch_dec1')(dense_dec_1)
    dense_relu_dec1 = Activation('relu', name='dense_relu_dec1')(dense_batch_dec1)
    dense_drop_dec1 = Dropout(0.3, name='dense_drop_dec1')(dense_relu_dec1)    
    
    dense_dec_2 = Dense(64, name='dense_dec_2')(dense_drop_dec1)
    dense_batch_dec2 = BatchNormalization(name='dense_batch_dec2')(dense_dec_2)
    dense_relu_dec2 = Activation('relu', name='dense_relu_dec2')(dense_batch_dec2)
    dense_drop_dec2 = Dropout(0.3, name='dense_drop_dec2')(dense_relu_dec2)
    
    key_hat = Dense(key_length, activation='softmax', name='key_hat')(dense_drop_dec2)
    
    
    model_decoder = Model(inputs=[Input_ipd], outputs=[key_hat])
    #model_decoder.set_weights(model.get_weights())
    #model_decoder.load_weights(filepath=path + model_name + ".h5", by_name=True)

    return model_decoder

def compute_extract_rate(keys, true_keys):
    correct = 0 
    for i in range(len(keys)): 
        if np.argmax(keys[i]) == np.argmax(true_keys[i]):
            correct +=1
    return correct/float(len(keys))



In [3]:
path = "/home/fatemeh/MyProjects/Fingerprint/models/"
rate = '10'
all_ipds_for_test = create_ipd_dataset(
    address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/' + rate + '/test/')
all_ipds_for_train = create_ipd_dataset(
    address='/home/fatemeh/MyProjects/Fingerprint/Synthetic dataset/in/' + rate + '/train/')



In [17]:
n_true_train, n_test = 100000, 5000
sample_size, key_length = 300, 1024
alpha = 25

X_train_all = create_sample_size_dataset(all_ipds_for_train, sample_size = sample_size,n_sample=n_true_train)
X_test_all = create_sample_size_dataset(all_ipds_for_test, sample_size = sample_size,n_sample=n_test)
print(len(X_train_all), len(X_test_all), "Numbre of training and testing data")



key_length = 100
key_options = selecting_valid_fingerprints(key_length = key_length)
X_train, y_train, train_keys = get_false_true_data(X_train_all, key_options, alpha=alpha)#get_only_true_data
train_keys = np.array(train_keys)
print("finished generating training data!...")

100000 5000 Numbre of training and testing data


In [4]:
import keras.backend as K
from keras import optimizers
import datetime
import time

def mean_pred_loss(y_true, y_pred):
    #when the coeficent is smaller, performance is better. When increasing, noise improves
    sum_abs = K.abs(y_pred)
    tmp =  K.mean(sum_abs) - 0.3 * K.mean(y_pred)# + K.epsilon()
    return 100 * (K.abs(K.mean(y_pred)))# 1/tmp  keras.losses.mean_absolute_error(y_true, y_pred) + 

def get_mult_sub_for_fingerprinting(n_data, max_delay, chunk, sample_size):
    array_mult, array_sub = [], []
    for x in range(0, n_data):
        array_mult.append([max_delay] * chunk)
        array_sub.append([-max_delay/2] * chunk)
    array_mult = np.array(array_mult)
    array_sub = np.array(array_sub)
    return array_mult, array_sub

def get_noise_simulation_array(n_data, std, sample_size):
    noise = []
    for x in range(0, n_data):
        #noise.append(np.random.normal(0, std, sample_size))
        #noise.append(np.random.uniform(0, std, sample_size))
        noise.append(np.random.laplace(0, std, sample_size));
    noise = np.array(noise)
    return noise
n_trains = [200000]
key_len = [512, 1024, 1024 *4, 1024*32]#2^(8, 10, 12, 15)
models = []
for key_length in key_len:
        for n in n_trains:
        
            n_true_train, n_test = n, 5000
            alpha = 25

            X_train_all = create_sample_size_dataset(all_ipds_for_train, sample_size = sample_size,n_sample=n_true_train)
            X_test_all = create_sample_size_dataset(all_ipds_for_test[500*300:], sample_size = sample_size,n_sample=n_test)
            print(len(X_train_all), len(X_test_all), "Numbre of training and testing data")

            key_options = selecting_valid_fingerprints(key_length = key_length)
            X_train, y_train, train_keys = get_false_true_data(X_train_all, key_options, alpha=alpha)#get_only_true_data
            train_keys = np.array(train_keys)
            print("finished generating training data!...")


            beg_time = time.time()
            x_fing_w, key_hat_w, epoch, batch = 1, 100, 50, 64
            std, max_fing_delay = 1, alpha
            chunk = 10

            array_mult_test, array_sub_test =  get_mult_sub_for_fingerprinting(n_test, max_delay=max_fing_delay, chunk=chunk,
                                                                                    sample_size=sample_size)
            noise_for_test = get_noise_simulation_array(n_test, std=std, sample_size=sample_size)
            array_mult_train, array_sub_train =  get_mult_sub_for_fingerprinting(n_true_train, max_delay=max_fing_delay, chunk=chunk,
                                                                                    sample_size=sample_size)
            noise_for_train = get_noise_simulation_array(n_true_train, std=std, sample_size=sample_size)
            t = n_true_train

            print("Date : ", datetime.datetime.now())

            model_10 = get_encoder_decoder_conv_dense_slice(sample_size=sample_size, key_length=key_length, chunk=chunk)
            # losses.mean_squared_error# 0.001
            ad = optimizers.Adam(lr = 1e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad=False)

            model_10.compile(optimizer=ad, loss={'fingerprint': mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                      loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})

            # model.summary()
            print("Model %s is Built and Compiled in %f" % (t, time.time() - beg_time))
            beg_time = time.time()

            model_10.fit([X_train[0:t], train_keys[0:t], array_mult_train[0:t], array_sub_train[0:t], noise_for_train[0:t]],
                  [y_train[0:t], train_keys[0:t]], epochs=epoch, validation_split=0.1,
                  batch_size=batch)  # callbacks=callbacks_list, verbose=0)
            models.append(model_10)

            print("Time to Fit the Model", time.time() - beg_time)

            #### This is when we test encoder and decoder together using the same model: model_encoder_decoder
            x_test = np.array(X_test_all[0:n_test]).reshape((-1, sample_size, 1))
            key_options = selecting_valid_fingerprints(key_length=key_length)  # we use 100 keys.
            test_keys = np.array(get_keys_for_fingerprinting_data(size=n_test, key_options=key_options))
            noise_for_test = np.squeeze(noise_for_test)
            pred = model_10.predict([x_test, test_keys, array_mult_test, array_sub_test, noise_for_test])

            fingerprint_x22, keys_true = pred[0], pred[1]
            ext_rate = compute_extract_rate(keys_true, true_keys=test_keys)

            print("Ext Rate:  ", ext_rate)
# avg_d, max_d = compute_delay_on_each_packet(fingerprint_x2)

#fingerprint_x2 = adjust_fingerprint_delays(fingerprint_x2)
#avg_d, max_d = compute_delay_on_packets(fingerprint_x2)

NameError: name 'sample_size' is not defined

In [ ]:
Time to Fit the Model 22027.91614151001, is ~6 hours.
Ext Rate:   1.0 when 1000000 traiing wiht key_length = 500

Same thing with key_length = 1000 is : 
    
#### try the sample_size = 600....

In [ ]:
#alpha =25
#pkt = 10,  ext_rate = 0.701, key_length = 20, epoch = 100, n_train = 5000
#usiing uniform (0,alpha) for fingerprint same parameters as above line: ext_rate: 0.54

Time to Fit the Model 2291.7441816329956 = 2291/3600 = 36 minutes?
Ext Rate:   0.978 with 10000 training and 20 keys, with 20,000 it goes to ext_rate = 1
    10000 training and 200 key  ext_rate =0.019
    with 20000 and 200 key ext_rate =0.53
    and with 40, 000 it goes to ext_rate = 1
.

#n_train = 24000 with key_length = 200, epoch = 50, pkt  =100 , ext_rate = 1

In [5]:
def get_fingerprints_for_ipds(n_train, sample_size, alpha):
    
    #Previous one was all + and the largest value was 50.
   
    fingerprint_output = []
    thrshold = 100
    while len(fingerprint_output) < n_train:
        finger = [0]#[random.uniform(0, 250)]
        neg_numbers = 0
        delay = 0
        for i in range(sample_size - 1):
            #if random.randrange(0, 3) == 0:
            rnd = random.randrange(0, 2)
            if rnd == 1 and delay < thrshold:
                finger.append(alpha)
            elif rnd ==1 and delay>= thrshold:
                finger.append(-alpha)
                
            elif rnd == 0 and delay <= 0 and  delay >-thrshold: # random.randrange(0, 2) == 0:# and np.abs(delay) <50:
                finger.append(-1 * alpha)#
            elif rnd ==0 and delay <=0 and delay <=-thrshold:
                finger.append(alpha)
            elif rnd == 0 and delay > 0:
                finger.append(-1 * alpha)
            delay += finger[-1]
            #print(delay)
            if sum(finger) < 0:
                neg_numbers += 1
#             else:
#                 finger.append(0) 
        #if neg_numbers < 50:  ## this can be a hyperparameter
        fingerprint_output.append(finger)
    return fingerprint_output



y = get_fingerprints_for_ipds(1, 1500, alpha=25) 
y = np.expand_dims(y, axis=1).reshape((-1, 1500, 1))
y = adjust_fingerprint_delays(y)
avg_d, max_d = compute_delay_on_packets(y)

NameError: name 'adjust_fingerprint_delays' is not defined

In [12]:
def adjust_fingerprint_delays(fingerprint_x2):
    number_of_train, sample_size = len(fingerprint_x2), len(fingerprint_x2[0])
    for f in range(0, len(fingerprint_x2)):
            delay, min_delay = 0, 0
            for p in range(0, len(fingerprint_x2[f])):
                    delay += fingerprint_x2[f][p][0]
                    #print(delay)
                    if delay < min_delay:
                        min_delay = delay 
            fingerprint_x2[f][0][0] -= 1.001 * min_delay
           # print("min",min_delay)
           # break
    return fingerprint_x2

def compute_delay_on_packets(fingerprint_x2):
#######Compute the average delay on each packet.        
    average_delay = []
    max_d, pos = 0, 0
    for fing in fingerprint_x2:
        delay, neg = 0, 0
        delays = []
        for n in fing:
            delay += n[0]
            if delay > max_d:
                max_d = delay
            delays.append(delay)

            if delay < 0:
                neg += 1
        if neg == 0:
            pos += 1
            average_delay.append(sum(delays)/sample_size)
   # print(sum(delays)/sample_size," Average Delay")
   # print(sample_size, number_of_train)
    print("Negative delay: ",pos, "Average delay for them:",sum(average_delay)/pos, "Max Delay: ", max_d)
    return sum(average_delay)/pos, max_d
fingerprint_x2 = adjust_fingerprint_delays(fingerprint_x2)
avg_d, max_d = compute_delay_on_packets(fingerprint_x2)


Negative delay:  4000 Average delay for them: 421.7638539961676 Max Delay:  2091.979405403137


In [12]:
for i in range(100):
    target = open("/home/fatemeh/MyProjects/Fingerprint/encoder/fing_ipds/" + str(i) + ".txt", 'w')
    array = x_test[i] + fingerprint_x2[i]
    array = np.squeeze(array)
    write_array_to_file(array, target, " ")

In [14]:
# TODO: I want to check to see if I 0 the negative fingerprints, I can have the same extraction rate.
model_decoder = load_decoder(key_length, sample_size)
#model_decoder.set_weights(model.get_weights())
beg_time = time.time()
i = 0
for layer in model_10.layers:
  #  w_target=layer.get_weights()
    for dec_layer in model_decoder.layers:
        if layer.name ==dec_layer.name:
            print(layer.name, i)
            dec_layer.set_weights(layer.get_weights())
    i += 1
print("Time it takes to load the decoder: ", time.time() - beg_time)

reshape_dec 1960
conv_dec_2 1961
conv_batch_2_dec 1962
conv_relu_2_dec 1963
conv_drop_2_dec 1964
max_pool_dec_2 1965
conv_dec_3 1966
conv_batch_3_dec 1967
conv_relu_3_dec 1968
conv_drop_3_dec 1969
max_pool_dec_3 1970
flate_max3_dec 1971
dense_dec_1 1972
dense_batch_dec1 1973
dense_relu_dec1 1974
dense_drop_dec1 1975
dense_dec_2 1976
dense_batch_dec2 1977
dense_relu_dec2 1978
dense_drop_dec2 1979
key_hat 1981
Time it takes to load the decoder:  126.30563545227051


In [31]:
def read_fingerprinted_ipds():
    path = '/home/fatemeh/MyProjects/Fingerprint/encoder/ext_ipds/'
    all_ipds = []
    for i in range(11):
        string_ipds = read_from_file(path + str(i) + ".txt").split(" ")
        ipds = convert_stringArrays_to_floatArray(string_ipds)
        all_ipds.append(ipds)
    return all_ipds
all_ipds = read_fingerprinted_ipds()
fingerprinted_ipds=np.array(all_ipds).reshape((-1, sample_size, 1))

In [39]:
'''
noise_for_test = np.squeeze(noise_for_test)
noise_for_test = get_noise_simulation_array(n_test, std=1, sample_size=sample_size)
noise_for_test=np.array(noise_for_test[0:n_test]).reshape((-1, sample_size, 1))
fingerprinted_ipds = fingerprint_x2 + x_test + noise_for_test
'''


key_hat = model_decoder.predict(fingerprinted_ipds)
ext_rate = compute_extract_rate(key_hat, true_keys=test_keys)
print(ext_rate)

1.0


In [29]:
import csv
# avg_d, max_d = compute_delay_on_each_packet(fingerprint_x2)
rate=100
with open('sep_results.csv', mode='a') as csv_file:
    fieldnames = ['sample_size', 'key_length', "number_training", 'ext_rate', 'average_delay', 'max_delay',
                  'packet_rate', 'std', 'max_train_noise','date']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    #writer.writeheader()
    writer.writerow(
        {'sample_size': sample_size, 'key_length': key_length, "number_training": n_true_train, 'ext_rate': ext_rate,
         'average_delay': avg_d, 'max_delay': max_d, 'packet_rate': rate, 'std': std, 'max_train_noise': max_fing_delay,'date': datetime.datetime.now()})

In [57]:
#print(X_train[0][0:100])
# print(fingerprint_x2[0][0:100])
array_mult_test2, array_sub_test2, noise_for_test2 = get_arrays_mult_noise_sub(10,max_delay=5,chunk=10,std=5,sample_size=sample_size)

d = 0
import numpy as np
for f in noise_for_test2:
    dd = []
    for p in f:
        dd.append(p)
    std = np.std(dd, axis=0)
    print(std)



4.993533930040474
4.925919764672603
4.863956283428593
4.892161564091119
5.108175177722096
4.905545496833845
4.938426689987192
4.921194612130275
4.9018388531082415
4.888976889759503


In [10]:
ext_rate = compute_extract_rate(keys_true, true_keys = test_keys)

print("Ext Rate:  ", ext_rate)
compute_delay_on_each_packet(fingerprint_x2)

Ext Rate:   0.0195
98.70847329990069
1500 4000
4000 49.32459005435246 1500 Max Delay:  223.82663989067078


In [14]:
import keras.backend as K
from keras import optimizers
n_false_train = 0
x_fing_w, key_hat_w, epoch, batch = 1, 200, 50, 64

beg_time = time.time()
key_length = 100
key_options = selecting_valid_fingerprints(key_length = key_length)
sample_sizes = [1500, 1800]#, 600, 1200]
#models = []
trains = [60000, 10000]
for sam in sample_sizes:
    sample_size = sam
    X_train_all = create_sample_size_dataset(all_ipds_for_train, sample_size = sample_size)
    X_test_all = create_sample_size_dataset(all_ipds_for_test, sample_size = sample_size)
    print(len(X_train_all),len(X_test_all), "Numbre of training and testing data")
    model = get_encoder_decoder_conv_dense_slice(sample_size=sample_size, key_length=key_length, chunk=10)
    ad = optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad=False)

    model.compile(optimizer=ad, loss={'fingerprint':mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                                    loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})
    X_train, y_train, train_keys = get_false_true_data(X_train_all, key_options)#get_only_true_data
    train_keys = np.array(train_keys)
    n_test = 4000
    for t in trains:
        # model.summary()
        beg_time = time.time()
        array_mult_train, array_sub_train, noise_for_train = get_arrays_mult_noise_sub(t,max_delay=5,chunk=10,std=1,sample_size=sample_size)
        model.fit([X_train[0:t], train_keys[0:t], array_mult_train[0:t], array_sub_train[0:t], 
                   noise_for_train[0:t]], [y_train[0:t], train_keys[0:t]], epochs=epoch,
                  validation_split=0.1, batch_size=batch,verbose =1)#, validation_split=0.1,callbacks=callbacks_list, verbose=0)

        print("Time to Fit the Model", time.time() - beg_time)
        models.append(model) 
        array_mult_test, array_sub_test, noise_for_test = get_arrays_mult_noise_sub(n_test,max_delay=5,chunk=10,std=1,sample_size=sample_size)

        #### This is when we test encoder and decoder together using the same model: model_encoder_decoder
        x_test = np.array(X_test_all[0:n_test]).reshape((-1, sample_size, 1))
        key_options = selecting_valid_fingerprints(key_length = key_length)# we use 100 keys.
        test_keys = np.array(get_keys_for_fingerprinting_data(size=n_test, key_options=key_options))
        noise_for_test = np.squeeze(noise_for_test[0:n_test])
        pred = model.predict([x_test, test_keys, array_mult_test, array_sub_test, noise_for_test])

        fingerprint_x2, keys_true = pred[0],  pred[1]
        ext_rate = compute_extract_rate(keys_true, true_keys = test_keys)

        print("Ext Rate:  ", ext_rate)
        compute_delay_on_each_packet(fingerprint_x2)

81200 5333 Numbre of training and testing data
100 ratio
finished getting true data
Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 525s 10ms/step - loss: 940.4309 - fingerprint_loss: 0.5370 - key_hat_loss: 4.6995 - val_loss: 919.3937 - val_fingerprint_loss: 0.7712 - val_key_hat_loss: 4.5931
Epoch 2/50
54000/54000 [==============================] - 260s 5ms/step - loss: 900.0707 - fingerprint_loss: 0.5327 - key_hat_loss: 4.4977 - val_loss: 886.4092 - val_fingerprint_loss: 0.4663 - val_key_hat_loss: 4.4297
Epoch 3/50
54000/54000 [==============================] - 259s 5ms/step - loss: 758.0978 - fingerprint_loss: 0.4542 - key_hat_loss: 3.7882 - val_loss: 900.9127 - val_fingerprint_loss: 0.4163 - val_key_hat_loss: 4.5025
Epoch 4/50
54000/54000 [==============================] - 269s 5ms/step - loss: 310.8727 - fingerprint_loss: 0.3480 - key_hat_loss: 1.5526 - val_loss: 175.3338 - val_fingerprint_loss: 1.0500 - val_key_hat_loss: 0

9000/9000 [==============================] - 44s 5ms/step - loss: 11.4743 - fingerprint_loss: 0.3522 - key_hat_loss: 0.0556 - val_loss: 3.2145 - val_fingerprint_loss: 2.1029 - val_key_hat_loss: 0.0056
Epoch 26/50
9000/9000 [==============================] - 44s 5ms/step - loss: 11.8253 - fingerprint_loss: 0.3625 - key_hat_loss: 0.0573 - val_loss: 2.6989 - val_fingerprint_loss: 2.2489 - val_key_hat_loss: 0.0023
Epoch 27/50
9000/9000 [==============================] - 44s 5ms/step - loss: 10.0726 - fingerprint_loss: 0.3354 - key_hat_loss: 0.0487 - val_loss: 2.4329 - val_fingerprint_loss: 2.1389 - val_key_hat_loss: 0.0015
Epoch 28/50
9000/9000 [==============================] - 44s 5ms/step - loss: 10.8778 - fingerprint_loss: 0.3097 - key_hat_loss: 0.0528 - val_loss: 4.8458 - val_fingerprint_loss: 2.4681 - val_key_hat_loss: 0.0119
Epoch 29/50
9000/9000 [==============================] - 44s 5ms/step - loss: 10.1352 - fingerprint_loss: 0.3204 - key_hat_loss: 0.0491 - val_loss: 3.2815 - val

Epoch 50/50
54000/54000 [==============================] - 340s 6ms/step - loss: 13.2627 - fingerprint_loss: 0.2737 - key_hat_loss: 0.0649 - val_loss: 7.4658 - val_fingerprint_loss: 1.8670 - val_key_hat_loss: 0.0280
Time to Fit the Model 18055.78747653961
Ext Rate:   0.99225
90.50788768331209
1800 4000
4000 84.35799002459503 1800 Max Delay:  371.3043042421341
Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 56s 6ms/step - loss: 14.3306 - fingerprint_loss: 0.2273 - key_hat_loss: 0.0705 - val_loss: 1.8057 - val_fingerprint_loss: 1.7722 - val_key_hat_loss: 1.6755e-04
Epoch 2/50
9000/9000 [==============================] - 54s 6ms/step - loss: 10.9617 - fingerprint_loss: 0.2458 - key_hat_loss: 0.0536 - val_loss: 1.5877 - val_fingerprint_loss: 1.5775 - val_key_hat_loss: 5.0716e-05
Epoch 3/50
9000/9000 [==============================] - 54s 6ms/step - loss: 9.3417 - fingerprint_loss: 0.2633 - key_hat_loss: 0.0454 - val_loss: 1.7548 - val

9000/9000 [==============================] - 60s 7ms/step - loss: 6.7460 - fingerprint_loss: 0.2804 - key_hat_loss: 0.0323 - val_loss: 2.1369 - val_fingerprint_loss: 2.1080 - val_key_hat_loss: 1.4490e-04
Epoch 38/50
9000/9000 [==============================] - 62s 7ms/step - loss: 5.9046 - fingerprint_loss: 0.2718 - key_hat_loss: 0.0282 - val_loss: 2.0500 - val_fingerprint_loss: 1.9125 - val_key_hat_loss: 6.8789e-04
Epoch 39/50
9000/9000 [==============================] - 62s 7ms/step - loss: 6.2920 - fingerprint_loss: 0.2684 - key_hat_loss: 0.0301 - val_loss: 2.0410 - val_fingerprint_loss: 2.0018 - val_key_hat_loss: 1.9621e-04
Epoch 40/50
9000/9000 [==============================] - 62s 7ms/step - loss: 6.0196 - fingerprint_loss: 0.2531 - key_hat_loss: 0.0288 - val_loss: 1.7909 - val_fingerprint_loss: 1.7572 - val_key_hat_loss: 1.6834e-04
Epoch 41/50
9000/9000 [==============================] - 60s 7ms/step - loss: 6.0426 - fingerprint_loss: 0.2643 - key_hat_loss: 0.0289 - val_loss: 1

'\n\n    size = 1800\n    Ext Rate:   0.992\n    4000 95.84566262555747 1800 Max Delay:  415.03377401828766\n    Ext Rate:   0.9765\n    4000 93.27346397158448 1800 Max Delay:  380.78301668167114\n    \n    \n    size 1200\n    \n    Ext Rate:   0.948\n    4000 72.26921833757295 1200 Max Delay:  357.55537247657776\n\n    Ext Rate:   0.943\n    4000 72.2022527100891 1200 Max Delay:  356.89986884593964\n    \n    size = 600\n    Ext Rate: 0.788\n    4000 38.018    max delay: 235.928\n    \n    size 600, and 10000 training:\n    Ext Rate:   0.731\n    4000 37.918288900979874 600 Max Delay:  233.056\n    \n\n'

In [11]:
n_test = 5000
print(sample_size)
array_mult_test, array_sub_test, noise_for_test = get_arrays_mult_noise_sub(n_test,max_delay=5,chunk=10,std=1,sample_size=sample_size)

x_test = np.array(X_test_all[0:n_test]).reshape((-1, sample_size, 1))
key_options = selecting_valid_fingerprints(key_length = 200)# we use 100 keys.
test_keys = np.array(get_keys_for_fingerprinting_data(size=n_test, key_options=key_options))
noise_for_test = np.squeeze(noise_for_test[0:n_test])
pred = model.predict([x_test, test_keys, array_mult_test, array_sub_test, noise_for_test])

fingerprint_x2, keys_true = pred[0],  pred[1]
ext_rate = compute_extract_rate(keys_true, true_keys = test_keys)

print("Ext Rate:  ", ext_rate)
compute_delay_on_each_packet(fingerprint_x2)

1500
Ext Rate:   0.8862
50.484288698991136
1500 5000
5000 77.23775366936363 1500 Max Delay:  401.2521390914917


'\nsample size = 1800, n_train = 10000, and key = 100:\n1800\nExt Rate:   0.0842\n52.90532826509741\n1800 4444\n4444 47.894631279740935 1800 Max Delay:  182.74153697490692\nwhen we only change n_train = 60000:\n\n\n\n'

In [3]:

def decide_if_fingerprinted(keys, threshold):
    fing = 0
    for key in keys:
        index = np.argmax(key)
        if key[index] > threshold and index > 0:
            fing += 1
    return fing / float(len(keys))

In [16]:
### Loading encoder takes too much time (hours), so we just use the model_encoder_decoder for encoding.
model_decoder = load_decoder(key_length, sample_size)
decoder_weights = []
j = 0
for i in range(0, 24):
    if 'dec' in model.layers[-(24 - i)].name or 'key_hat' in model.layers[-(24 - i)].name:
        model_decoder.layers[j].set_weights(model.layers[-(24 - i)].get_weights())
        j += 1

In [17]:
noise_for_test = noise_for_test.reshape((-1, sample_size, 1))

output_fin = noise_for_test[0:n_test] + x_test
keys_true_fp = model_decoder.predict([output_fin])

###### True positve:
output_fin = noise_for_test[0:n_test] + x_test + fingerprint_x2

keys_true_tp = model_decoder.predict([output_fin])
ext_rate = compute_extract_rate(keys_true_tp, test_keys)
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]


for t in thresholds:
    fp = decide_if_fingerprinted(keys_true_fp, t)
    tp = decide_if_fingerprinted(keys_true_tp, t)
    
    print(fp, tp)
print(ext_rate, 'Extraction Rate')
'''

key = 100 training with 1/100 number of false data. sample size = 1800, number of training=20000
0.2395 0.9845
0.156 0.9695
0.093 0.9535
0.043 0.931
0.014 0.8975
0.0055 0.867
0.0005 0.766
0.9685 Extraction Rate
Ext Rate:   0.969
1800 2000
2000 138.32507355565957 1800 Max Delay:  556.0820367336273
##############################################################################################


sample size 3300, training data = 48000, key = 1000

0.293 0.9925
0.1925 0.98
0.115 0.9685
0.0655 0.941
0.022 0.905
0.007 0.8525
0.0 0.734
0.9695 Extraction Rate
2000 80.0513701567251 3300 Max Delay:  369.6583148241043

'''

0.293 0.9925
0.1925 0.98
0.115 0.9685
0.0655 0.941
0.022 0.905
0.007 0.8525
0.0 0.734
0.9695 Extraction Rate


'\n\nkey = 100 training with 1/100 number of false data. sample size = 1800, number of training=20000\n0.2395 0.9845\n0.156 0.9695\n0.093 0.9535\n0.043 0.931\n0.014 0.8975\n0.0055 0.867\n0.0005 0.766\n0.9685 Extraction Rate\nExt Rate:   0.969\n1800 2000\n2000 138.32507355565957 1800 Max Delay:  556.0820367336273\n##############################################################################################\n\n\nsample size 3300, training data = 45000, key = 1000\n\n0.327 0.9895\n0.1985 0.9705\n0.122 0.9525\n0.0635 0.933\n0.025 0.9\n0.0095 0.861\n0.0015 0.7415\n0.9675 Extraction Rate\n\n'

In [ ]:
n_false_train = 0
x_fing_w, key_hat_w, epoch, batch = 1, 200, 100, 64
model_name = str(sample_size) + "_" + str(key_length) + "_" + str(
    n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_fing_w) + "_" + str(key_hat_w)

beg_time = time.time()
#models_key_length = []
keys = [100]
n_true_train = 30000
for k in keys:

    key_options = selecting_valid_fingerprints(key_length = k)# we use 100 keys.
    X_train, y_train, train_keys = get_false_true_training(X_train_all[0:n_true_train], key_options)
    train_keys = np.array(train_keys)
    print("Finished radinf")

    model= get_encoder_decoder_conv_dense_slice(sample_size=sample_size, key_length=k)
    #losses.mean_squared_error
    ad = optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad=False)

    model.compile(optimizer=ad, loss={'fingerprint':mean_pred_loss, 'key_hat': losses.categorical_crossentropy},
                                    loss_weights={'fingerprint': x_fing_w, 'key_hat': key_hat_w})


    # model.summary()
    print("Model %s is Built and Compiled in %f" % (model_name ,time.time() - beg_time))
    beg_time = time.time()

    model.fit([X_train, train_keys, array_mult_train[0:n_true_train], array_sub_train[0:n_true_train], noise_for_train[0:n_true_train]], [y_train, train_keys], epochs=epoch, validation_split=0.1, batch_size=batch)#, validation_split=0.1,callbacks=callbacks_list, verbose=0)

    print("Time to Fit the Model", time.time() - beg_time)

    
    models_key_length.append(model)




In [ ]:
path_for_results = '/home/fatemeh/Dropbox/Fingerprint/Results/'
def compute_ROC_data(n_train):
    target_name = open(path_for_results + str(n_train)+"_" + str(sample_size)+"_"+str(key_length) + '.txt', 'w')
    sample_size = 900
    key_length = 100
    n_test = 5000
    thresholds = [0.6, 0.7, 0.8, 0.9]
    X = create_sample_size_dataset(all_ipds, sample_size = sample_size)
    key_options = selecting_valid_fingerprints(key_length = key_length)

    


    model_decoder, model_encoder = load_model_for_testing(key_length, sample_size, n_train)
    X_test = np.expand_dims(X[n_train:n_train + n_test], axis=1)
    test_keys = np.expand_dims(get_fingerprint_for_data(size = n_test, key_options = key_options), axis=1)
    fingerprint_x = model_encoder.predict([test_keys])
    false_poses, true_poses = [], []
    
     ########## True positve:
    output_fin = add_gussian_noise_to_ipds_fingerprinted(fingerprint = fingerprint_x, x_test = X_test, std =10)
    keys_true = model_decoder.predict([output_fin])

    ########## False positve: 
    output_non = add_gussian_noise_to_ipds_non_fingerprinted(X_test, std =10)
    keys_false = model_decoder.predict([output_non])
    for t in thresholds:
        true_pos = decide_if_fingerprinted(keys_true, threshold = t)
        false_pos = decide_if_fingerprinted(keys_false, threshold = t)
        false_poses.append(false_pos)
        true_poses.append(true_pos)
        
    write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
    write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
    target_name.close()
    
compute_ROC_data(n_train=10000)

In [ ]:
def compute_impact_of_jitter():
    sample_size = 600
    key_length = 10
    n_train = 50000
    n_test = 5000
    X = create_sample_size_dataset(all_ipds, sample_size = sample_size)
    key_options = selecting_valid_fingerprints(key_length = key_length)

    target_name = open(path_for_results + str(n_train)+"_" + str(sample_size)+"_"+str(key_length) + '.txt', 'w')

    jitters = [1, 10, 50, 100]
   
    model_decoder, model_encoder = load_model_for_testing(key_length, sample_size, n_train)
    X_test = np.expand_dims(X[n_train:n_train + n_test], axis=1)
    test_keys = np.expand_dims(get_fingerprint_for_data(size = n_test, key_options = key_options), axis=1)
    fingerprint_x = model_encoder.predict([test_keys])
    false_poses, true_poses, ext_rates = [], [], []

    for std in jitters:      
        ########## True positve:
        output_fin = add_gussian_noise_to_ipds_fingerprinted(fingerprint = fingerprint_x, x_test = X_test, std =std)
        keys_true = model_decoder.predict([output_fin])
        true_pos = decide_if_fingerprinted(keys_true, threshold=4)
        key_pred = extract_keys_from_key_hat(keys_true)
        error_rate = compute_error_rate_flowwise(predict_key = key_pred, true_key = test_keys)

        ########## False positve: 
        output_non = add_gussian_noise_to_ipds_non_fingerprinted(X_test, std =std)
        keys_false = model_decoder.predict([output_non])
        false_pos = decide_if_fingerprinted(keys_false, threshold=4)
        false_poses.append(false_pos)
        true_poses.append(true_pos)
        ext_rates.append(1 - error_rate)
    write_array_to_file(array = ext_rates, target =target_name, delimiter =' ')
    write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
    write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
    target_name.close()
# compute_impact_of_jitter()

In [ ]:
# x_fing_w, key_hat_w, epoch = 1, 50, 100
def call_fit_load_eval_Main():
    n_all_true_trains =[5000]# [5000, 10000, 20000, 50000]#5000,, 
    sample_sizes = [600]#[400, 200, 600]
    key_lengths = [10]#, 15, 20]

    for sample_size in sample_sizes:
        X = create_sample_size_dataset(all_ipds, sample_size = sample_size)
        for key_length in key_lengths:
            key_options = selecting_valid_fingerprints(key_length = key_length)
            false_poses, true_poses, ext_rates = [], [], []
            target_name = open(path_for_results + str(sample_size)+"_"+str(key_length) + '.txt', 'w')

            for train_number in n_all_true_trains:
                false_pos, true_pos, ext_rate = fit_model_load_evaulte(n_true_train =train_number, key_length=key_length, sample_size=sample_size,X=X,key_options=key_options)
                false_poses.append(false_pos)
                true_poses.append(true_pos)
                ext_rates.append(ext_rate)
                print(false_pos, true_pos, ext_rate)
            write_array_to_file(array = ext_rates, target =target_name, delimiter =' ')
            write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
            write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
            target_name.close()
call_fit_load_eval_Main()

In [ ]:
def reload_model_for_more_epochs():
    sample_size, key_length, n_true_train, epoch = 600, 10, 50000, 250
    n_false_train = int(n_true_train/10)
    x_fing_w, key_hat_w = 1, 50
    model_name = "march_10" + str(sample_size) + "_" + str(key_length) + "_" + str(
    n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_fing_w) + "_" + str(key_hat_w)

    model = load_NN_model(path + model_name)
    model_encoder_decoder.compile(optimizer='adam', 
                              loss=losses.mean_absolute_error,
                                  loss_weights={'fingerprint':x_fing_w, 'key_hat':key_hat_w})

    model_encoder_decoder.fit([X_train, training_keys], [y_train, training_keys],
                        batch_size = 64, epochs = epoch + 250, verbose = 0)
    #save_model_weights(model_encoder_decoder, name=model_name)
# reload_model_for_more_epochs()

In [ ]:
def load_test_all_Main():
    n_all_true_trains = [10000]# 5000, 10000, 20000, 50000]
    sample_sizes = [600]
    key_lengths = [10]
    x_fing_w, key_hat_w, epoch = 1, 50, 100
    n_test = 1000
    for sample_size in sample_sizes:
            
            X = create_sample_size_dataset(all_ipds_for_test, sample_size = sample_size)
            for key_length in key_lengths:
                key_options = selecting_valid_fingerprints(key_length = key_length)
                false_poses, true_poses, ext_rates = [], [], []
                #target_name = open('/home/fatemeh/Dropbox/Fingerprint/Results/500_' + str(sample_size)+"_"+str(key_length) + '.txt', 'w')

                for train_number in n_all_true_trains:
                    model_decoder, model_encoder = load_model_for_testing(key_length, sample_size, train_number)
                    false_pos, true_pos, ext_rate = evalute_encoder_decoder(model_decoder,model_encoder, X, sample_size, key_length,
                                                                            key_options, train_number, int(train_number/10), n_test=n_test)
                    false_poses.append(false_pos)
                    true_poses.append(true_pos)
                    ext_rates.append(ext_rate)
                    print(sample_size, key_length, train_number, "Result: ", false_pos, true_pos, ext_rate)
#                 write_array_to_file(array = ext_rates, target =target_name, delimiter =' ')
#                 write_array_to_file(array = false_poses, target =target_name, delimiter =' ')
#                 write_array_to_file(array = true_poses, target =target_name, delimiter =' ')
#                 target_name.close() 
load_test_all_Main()

In [ ]:
n_true_trains = [5000, 10000, 20000, 50000, 100000]
sample_size, key_length = 600, 10
epoch = 250
for n_true_train in n_true_trains:
    n_false_train = int(n_true_train/10)
    key_hat_w, x_hat_w = 50, 1
    model_name = "march10_" + str(sample_size) + "_" + str(key_length) + "_" + str(
        n_true_train) + "_" + str(n_false_train) + "_" + str(epoch) + "_" + str(x_hat_w) + "_" + str(key_hat_w)

    model = load_NN_model(path + model_name)

    X_train, y_train, training_keys = get_false_true_training(n_true_train, n_false_train, key_length, X, key_options)
    print("Finished reading dataset")

    model.compile(optimizer='adam', 
                                  loss=losses.mean_absolute_error,
                                      loss_weights={'fingerprint':1, 'key_hat':50})
    model.fit([X_train, training_keys], [y_train, training_keys],
                            batch_size = 64, epochs = epoch, verbose = 0)
    
    model_name = "march10_" + str(sample_size) + "_" + str(key_length) + "_" + str(
        n_true_train) + "_" + str(n_false_train) + "_" + str(500) + "_" + str(x_hat_w) + "_" + str(key_hat_w)

    save_model_weights(model, name= model_name)


In [ ]:
def load_encoder(key_length, sample_size):
    chunk, p = 10, 0
    Input_ipd = Input(shape=(sample_size, 1), name='input1')  # this is needed just for the decoding
    Input_key = Input(shape=(key_length,), name='input2')
    fingerprint_mult = Input(shape=(chunk,), name='input3')
    fingerprint_sub = Input(shape=(chunk,), name='input4')
    
    ipd = Flatten(name ="ipd_flatten1")(Input_ipd)
    outputs = []
    
    quant = int(sample_size/chunk)
    def slice(x):
        return x[:, p * chunk:(1 + p) * chunk]
    
    key1 = Dense(32, name='key1')(Input_key)

    sliced_ipd = Lambda(slice)(ipd)
    x_fingerprint = sliced_ipd
    for i in range(0, quant):
        sliced_ipd = Lambda(slice)(ipd)
        ss = Concatenate(name = 'concat'+ str(p))([x_fingerprint, sliced_ipd]) 
        ipd1 = Dense(32, name = 'dense'+ str(p))(ss)
        batch_2 = BatchNormalization(name = 'batch'+ str(p))(ipd1)
        relu_2 = Activation('relu', name = 'act'+ str(p))(batch_2)
        
        ipds_merged_all = Concatenate(name = 'concat_key_'+ str(p))([relu_2, key1])
        dense_enc1 = Dense(64, name = 'dense_enc1' + str(p))(ipds_merged_all)
        batch_2 = BatchNormalization(name = 'batch2_'+ str(p))(dense_enc1)
        relu_2 = Activation('relu', name = 'act2_'+ str(p))(batch_2)
        dense_drop_enc1 = Dropout(0.3, name = 'dense_drop_enc1' + str(p))(relu_2)
        
        x_fingerprint_sig = Dense(chunk, name = 'fingerprint_sig' + str(p), activation = 'sigmoid')(dense_drop_enc1)
        x_fingerprint_mult = Multiply(name = 'fingerprint_mult' + str(p))([x_fingerprint_sig, fingerprint_mult])
        x_fingerprint = Add(name = 'ipd_delay' + str(p))([x_fingerprint_mult, fingerprint_sub])
        outputs.append(x_fingerprint)
        p += 1
    x_fingerprint = Concatenate(name = 'fingerprint2')(outputs)
    x_fingerprint_output = Reshape((sample_size,1), name='fingerprint')(x_fingerprint)
    model_encoder = Model(inputs=[Input_key,Input_ipd,  fingerprint_mult, fingerprint_sub], outputs=[x_fingerprint_output])
    #model_encoder.load_weights(filepath=path + model_name + ".h5", by_name=True)
    return model_encoder
